In [1]:
#import standard library modules
import sys
import re
import json
from collections import Counter
from typing import List, Any

In [2]:
#import the cmd2 modules
import os
import pprint
import cmd2
from cmd2 import style, fg, bg, Settable
import sys
import datetime
import json
from webbrowser import open_new_tab
import itertools as it
import BTCInput4 as btc
from app import wrapStringInHTMLMac3
#os.getcwd()

In [3]:
#import lexnlp
#import lexnlp.extract.en.entities.nltk_re #entity extraction

#modules from the community
import spacy
from dataclasses import dataclass #backported this module from 3.7

#import spacy and blackstone

from spacy import displacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler



from blackstone.displacy_palette import ner_displacy_options

#blackstone improved citations

from blackstone.pipeline.sentence_segmenter import SentenceSegmenter
from blackstone.pipeline.abbreviations import AbbreviationDetector
from blackstone.pipeline.compound_cases import CompoundCases
from blackstone.utils.legislation_linker import extract_legislation_relations
from blackstone.rules import CITATION_PATTERNS

from blackstone.pipeline import Concepts
from blackstone.rules import CONCEPT_PATTERNS

In [4]:
import lexnlp

In [5]:
#create cmd2 module, with the namespace having the locations of each file of data embedded. 
#let's add them to the entityruler
#keys, values:: companies: location_of_companies_, states: location_of_states, etc

In [6]:
#attribute self.model_loaded = False, but becomes true after a model is loaded

In [7]:
default_path = '/Users/thomassullivan/projects/GitHub/bulk_scotus/'

In [8]:
'''Note: not all of these commands will end up being included in the final
command line interface.

Note for 2/7/20: Add WrappedHTMLFormat properties to each class that we have created.'''

class ScotusCMD(cmd2.Cmd):
    '''
    The actual content of the commands should be moved to the app.py file.
    '''
    #prompt='ScotusSearch'
    #intro='Welcome to ScotusSearch'
    def __init__(self):
        super().__init__(multiline_commands=['add', 'export'], use_ipython=True)
        self.disable_command('quit', message_to_print='Enter "exit" to quit')
        self.intro = cmd2.style('Welcome to ScotusSearch and cmd2!',fg=cmd2.fg.red, bg=cmd2.bg.white, bold=True) + ' 😀'
        self.prompt = '(ScotusSearch)'
        self.self_in_py = True
        self.allow_style='Always'
        self.foreground_color = 'black'
        self.background_color = 'white'
        #self.__start_date = parse('05/01/2020').date()
        #self.__end_date = date.today()
        
        #new_app=app.App()
        #new_app.setup()
        #self.app=new_app
        #self.session=new_app.d.Session()
        self.add_settable(Settable('foreground_color', str,
                          'Foreground color to use with echo command',
                          choices=fg.colors()))
        self.add_settable(cmd2.Settable('background_color', str,
                          'Background color to use with echo command',
                          choices=bg.colors()))
        #self.session = dal.
        
    def do_hello(self, line):
        print('hello ', line)
    
    def do_search_year(self, line):
        try:
            os.chdir(default_path)
            cent_values = {17: '/1700s', 18:'/1800s', 19:'/1900s', 20: '/2000s'}
            try:
                assert len(line) == 4, 'the year is four digits long'
                assert line.isnumeric() == True, 'must enter a year'
            except AssertionError:
                print('command entered incorrectly')
            try:
                century = int(line[:2]) #make it into an integer to get the key
            except ValueError:
                print('Incorrect year format')
            path = os.getcwd()
            path = path + cent_values.get(century)
            #print('the new path is', path)
            path2=path + '/'+line
            print('the path of the individual year is: {0}'.format(path2))
            #print('these cases were decided that year:', os.listdir(path2))
            cases = [i for i in os.listdir(path2)]
            #cases = None
            dir_file = path2+'/year_directory.json'
            with open(dir_file) as f:
                case_dict = json.load(f)
            for k, v in case_dict.items():
                print(k, v['id'], v['date'], v['judges'])
            choice = btc.read_text('choose a case to view in html or "." to cancel: ')
            try:
                choice = case_dict[choice]['id']
            except KeyError:
                print('Key not found, is this what you are looking for?')
                #potentials = any(key.startswith(choice) for key in case_dict)
                choice = next( (key for key in case_dict if key.startswith(choice)), None )
                print(choice)
                continue_choice = btc.read_int_ranged(f'Continue with {choice}? 1-yes, 2-no', 1, 2)
                if continue_choice == 2:
                    print('Search cancelled, return to main menu')
                    return
                else:
                    choice = case_dict[choice]['id']
                #return
            if choice == ".":
                print('search cancelled, returning to main menu')
                return
            filename = choice + '.json'
            if filename in cases:
                os.chdir(path2)
                print(os.getcwd())
                with open(filename) as in_file:
                    print(filename)
                    new_case = json.load(in_file)
                #print('new case:' ,new_case.keys())
                filename_choice = btc.read_text('enter filename to save or "." to cancel: ')
                if filename_choice == '.':
                    print('file creation cancelled')
                    os.chdir(default_path)
                    return
                else:
                    wrapStringInHTMLMac3(program=filename_choice,
                                         body=new_case['html_with_citations'],
                                         url='test', default_path=default_path)
                    os.chdir(default_path)
            
        except UnboundLocalError:
            print('year entered incorrectly')
            
    def do_analyze_case(self, args):
        try:
            os.chdir(default_path)
            cent_values = {17: '/1700s', 18:'/1800s', 19:'/1900s', 20: '/2000s'}
            try:
                assert len(args.line) == 4, 'the year is four digits long'
                assert args.line.isnumeric() == True, 'must enter a year'
            except AssertionError:
                print('command entered incorrectly')
            try:
                century = int(args.line[:2]) #make it into an integer to get the key
            except ValueError:
                print('Incorrect year format')
            path = os.getcwd()
            path = path + cent_values.get(century)
            #print('the new path is', path)
            path2=path + '/'+args.line
            print('the path of the individual year is: {0}'.format(path2))
            #print('these cases were decided that year:', os.listdir(path2))
            cases = [i for i in os.listdir(path2)]
            #cases = None
            dir_file = path2+'/year_directory.json'
            with open(dir_file) as f:
                case_dict = json.load(f)
            for k, v in sorted(case_dict.items(), key=k):
                print(k, v['id'], v['date'], v['judges'])
            choice = btc.read_text('choose a case to view in html or "." to cancel: ')
            try:
                choice = case_dict[choice]['id']
            except KeyError:
                print('Key not found, is this what you are looking for?')
                #potentials = any(key.startswith(choice) for key in case_dict)
                choice = next( (key for key in case_dict if key.startswith(choice)), None )
                print(choice)
                continue_choice = btc.read_int_ranged(f'Continue with {choice}? 1-yes, 2-no', 1, 2)
                if continue_choice == 2:
                    print('Search cancelled, return to main menu')
                    return
                else:
                    choice = case_dict[choice]['id']
                #return
            if choice == ".":
                print('search cancelled, returning to main menu')
                return
            filename = choice + '.json'
            if filename in cases:
                os.chdir(path2)
                print(os.getcwd())
                with open(filename) as in_file:
                    print(filename)
                    new_case = json.load(in_file)
                #print('new case:' ,new_case.keys())
                filename_choice = btc.read_text('enter filename to save or "." to cancel: ')
                if filename_choice == '.':
                    print('file creation cancelled')
                    os.chdir(default_path)
                    return
                else:
                    wrapStringInHTMLMac3(program=filename_choice,
                                         body=new_case['html_with_citations'],
                                         url='test', default_path=default_path)
                    os.chdir(default_path)
            
        except UnboundLocalError:
            print('year entered incorrectly')
    
#    def help_year_search(self):
#        print('year_search [year]')
#        print('e.g. year_search 1999')
    
    def do_EOF(self, line):
        return True
    
    def do_exit(self, arg):
        #del arg
        #a.close()
        print('Exiting Roundup Generator')
        sys.exit()

In [ ]:
ScotusCMD().cmdloop()

-f is not a recognized command, alias, or macro
/Users/thomassullivan/Library/Jupyter/runtime/kernel-d89aa50b-6253-4fe2-97e1-906e7d9b043c.json is not a recognized command, alias, or macro


Welcome to ScotusSearch and cmd2! 😀


 help



Documented commands (use 'help -v' for verbose/'help <topic>' for details):
alias  help     ipy    py            run_script  shell    
edit   history  macro  run_pyscript  set         shortcuts

Undocumented commands:
EOF  exit  hello  search_year



 search_year 2009


...

Key not found, is this what you are looking for?
Quanta Computer, Inc. v. LG Electronics, Inc.


Continue with Quanta Computer, Inc. v. LG Electronics, Inc.? 1-yes, 2-no 1


/Users/thomassullivan/projects/GitHub/bulk_scotus/2000s/2008
145800.json


enter filename to save or "." to cancel:  quanta_file
 quanta_file.json


quanta_file.json is not a recognized command, alias, or macro
